## Psychrometric Properties Calculation

In [16]:
import math

### Case 1
If given pressure(P in psi), dry bulb temperature (T in F), and relative humidity (RH in %)

In [17]:
def pws(T):
    """
    Determine the saturation pressure of water vapor at a given temperature, T.

    Parameters:
        T: Temperature in degrees Fahrenheit [F].
    Returns:  Saturation pressure of water vapor in [psia].
    """
    # Convert temperature to absolute temperature [Rankine]
    t_abs = T + 459.67

    # Initialize coefficients
    C = [0] * 8  # Array to hold coefficients C(1) through C(7), indexed from 1

    # Set coefficients based on temperature range
    if -148 < T < 32:
        C[1] = -10214.165
        C[2] = -4.8932428
        C[3] = -0.0053765794
        C[4] = 1.9202377E-07
        C[5] = 3.5575832E-10
        C[6] = -9.0344688E-14
        C[7] = 4.1635019
    elif 32 <= T < 392:
        C[1] = -10440.397
        C[2] = -11.29465
        C[3] = -0.027022355
        C[4] = 1.289036E-05
        C[5] = -2.4780681E-09
        C[6] = 0
        C[7] = 6.5459673
    else:
        raise ValueError("Temperature out of valid range (-148 < T < 392)")

    # Determine the saturation pressure of saturated water
    pws = math.exp(C[1] / t_abs + C[2] + C[3] * t_abs + C[4] * t_abs**2 +
              C[5] * t_abs**3 + C[6] * t_abs**4 + C[7] * math.log(t_abs))

    return pws

In [18]:
def W_ptr(p, T, r):
    """
    Determine the humidity ratio for air/water vapor.

    Parameters:
        p (float): Absolute pressure of the mixture [psia].
        T (float): Dry bulb temperature [F].
        r (float): Relative humidity [%].

    Returns:
        W (float): Humidity ratio [lb_water/lb_dry_air].
    """
    # Determine the partial pressure of water vapor in the air, [psia]
    pw = (r / 100) * pws(T)

    # Calculate the humidity ratio
    W_ptr = 0.621945 * pw / (p - pw)
    return W_ptr

In [19]:
def Tdp_ptr(p, T, r):
    """
    Determine the dew point temperature for air/water vapor.

    Parameters:
        p (float): Absolute pressure of the mixture [psia].
        T (float): Dry bulb temperature [F].
        r (float): Relative humidity [%].

    Returns:
        float: Dew point temperature [F].
    """
    # Initialize coefficients
    C = [0, 100.45, 33.193, 2.319, 0.17074, 1.2063]

    # Determine the partial pressure of water vapor in the air, [psia]
    pw = (r / 100) * pws(T)
    alpha = math.log(pw)

    # Initial dew point temperature estimate
    Tdp_test = 90.12 + 26.142 * alpha + 0.8972 * alpha**2

    if Tdp_test < 32:
        T_dp =  Tdp_test
        return T_dp
    elif 32 <= Tdp_test <= 200:
        T_dp =  (C[1] + C[2] * alpha + C[3] * alpha**2 +
                C[4] * alpha**3 + C[5] * pw**0.1984)
        return T_dp
    else:
        raise ValueError("Dew point temperature out of valid range")

In [20]:
def h_ptr(p, T, r):
    """
    Determine the enthalpy for air/water vapor.

    Parameters:
        p (float): Absolute pressure of the mixture [psia].
        T (float): Dry bulb temperature [F].
        r (float): Relative humidity [%].

    Returns:
        h_ptr (float): Enthalpy [Btu/lb_da].
    """
    # Determine the humidity ratio
    W = W_ptr(p, T, r)

    # Calculate the enthalpy
    h_ptr = 0.24 * T + W * (1061 + 0.444 * T)
    return h_ptr

In [21]:
def h_ptw(p, T, W):
    """
    Determine the enthalpy for air/water vapor.

    Parameters:
        p (float): Absolute pressure of the mixture [psia].
        T (float): Dry bulb temperature [F].
        r (float): Relative humidity [%].

    Returns:
        h_ptr (float): Enthalpy [Btu/lb_da].
    """

    # Calculate the enthalpy
    h_ptw = 0.24 * T + W * (1061 + 0.444 * T)
    return h_ptw

In [22]:
def v_ptr(p, T, r):
    """
    Determine the specific volume for air/water vapor.

    Parameters:
    p (float): Absolute pressure of the mixture [psia]
    T (float): Dry-bulb temperature [F]
    r (float): Relative humidity [%]

    Returns:
    float: Specific volume [ft^3/lb_da]
    """
    # Define constants
    Rda = 53.34  # [ft*lbf/lbm*R]

    # Convert dry-bulb temperature to absolute temperature
    t_abs = T + 459.67

    # Calculate the partial pressure of water vapor (requires pws function)
    pw = (r / 100) * pws(T)  # pws(T) needs to be defined elsewhere

    # Calculate specific volume
    v = Rda * t_abs / (144 * (p - pw))

    return v

In [23]:
def Twb_ptr(p, T, r):
    """
    Determine the wetbulb temperature for air/water vapor.

    Parameters:
    p (float): Absolute pressure of the mixture [psia]
    T (float): Drybulb temperature [F]
    r (float): Relative humidity [%]

    Returns:
    float: Wetbulb temperature [F]
    """
    def Twb_res(p, T, r, Twb):
        """Residual function used in Newton's method."""
        if Twb >= 32:
            return -W_ptr(p, T, r) + ((1093 - 0.556 * Twb) * W_ptr(p, Twb, 100) - 0.24 * (T - Twb)) / (1093 + 0.444 * T - Twb)
        else:
            return -W_ptr(p, T, r) + ((1220 - 0.04 * Twb) * W_ptr(p, Twb, 100) - 0.24 * (T - Twb)) / (1220 + 0.444 * T - 0.48 * Twb)

    def Twb_res_deriv(p, T, r, Twb):
        """Derivative of the residual function with respect to Twb."""
        deltaTwb = 0.1  # [F]
        return (Twb_res(p, T, r, Twb + deltaTwb) - Twb_res(p, T, r, Twb - deltaTwb)) / (2 * deltaTwb)

    # Initial guess and tolerance for Newton's method
    Twb_guess = T
    tolerance = 1e-6

    # Calculate residual
    Residual = Twb_res(p, T, r, Twb_guess)

    # Newton's method loop
    while abs(Residual) > tolerance:
        Twb_next = Twb_guess - Twb_res(p, T, r, Twb_guess) / Twb_res_deriv(p, T, r, Twb_guess)
        Twb_guess = Twb_next
        Residual = Twb_res(p, T, r, Twb_guess)

    return Twb_guess

### Case 2
If give pressure(P in psi), dry bulb temperature (T in F), and wet bulb temperature (B in F)

In [24]:
def W_ptb(p: float, T: float, B: float) -> float:
    """
    Determine the humidity ratio for air/water vapor.
    
    Parameters:
    p (float): Absolute pressure of the mixture [psia]
    T (float): Drybulb temperature [F]
    B (float): Wetbulb temperature [F]
    
    Returns:
    float: Humidity ratio [lb_water/lb_da]
    """
    
    if B >= 32:
        W = ((1093 - 0.556 * B) * W_ptr(p, B, 100) - 0.24 * (T - B)) / (1093 + 0.444 * T - B)
    else:
        W = ((1220 - 0.04 * B) * W_ptr(p, B, 100) - 0.24 * (T - B)) / (1220 + 0.444 * T - 0.48 * B)
    return W

In [25]:
def RH_ptb(p: float, T: float, B: float) -> float:
    """
    Determine the relative humidity for air/water vapor.
    
    Parameters:
    p (float): Absolute pressure of the mixture [psia]
    T (float): Drybulb temperature [F]
    B (float): Wetbulb temperature [F]
    
    Returns:
    float: Relative humidity [%]
    """
    
    W = W_ptb(p, T, B)
    pw = (W / 0.621945) * p / (1 + (W / 0.621945))
    pwsat = pws(T)
    return (pw / pwsat) * 100

In [26]:
def RH_ptw(p: float, T: float, W: float) -> float:
    """
    Determine the relative humidity for air/water vapor.
    
    Parameters:
    p (float): Absolute pressure of the mixture [psia]
    T (float): Drybulb temperature [F]
    W (float): Humidity Ratio lbm/lbm
    
    Returns:
    float: Relative humidity [%]
    """
    
    pw = (W / 0.621945) * p / (1 + (W / 0.621945))
    pwsat = pws(T)
    return (pw / pwsat) * 100

In [27]:
def Tdp_ptb(p: float, T: float, B: float) -> float:
    """
    Determine the dewpoint temperature for air/water vapor.
    
    Parameters:
    p (float): Absolute pressure of the mixture [psia]
    T (float): Drybulb temperature [F]
    B (float): Wetbulb temperature [F]
    
    Returns:
    float: Dewpoint temperature [F]
    """   
    RH = RH_ptb(p, T, B)
    return Tdp_ptr(p, T, RH)

In [28]:
def h_ptb(p: float, T: float, B: float) -> float:
    """
    Determine the enthalpy for air/water vapor.
    
    Parameters:
    p (float): Absolute pressure of the mixture [psia]
    T (float): Drybulb temperature [F]
    B (float): Wetbulb temperature [F]
    
    Returns:
    float: Enthalpy [Btu/lb_da]
    """
    
    RH = RH_ptb(p, T, B)
    return h_ptr(p, T, RH)

In [29]:
def v_ptb(p: float, T: float, B: float) -> float:
    """
    Determine the specific volume for air/water vapor.
    
    Parameters:
    p (float): Absolute pressure of the mixture [psia]
    T (float): Drybulb temperature [F]
    B (float): Wetbulb temperature [F]
    
    Returns:
    float: Specific volume [ft^3/lb_da]
    """
    
    RH = RH_ptb(p, T, B)
    return v_ptr(p, T, RH)

In [30]:
def Tdp_ptW(p, T, W):
    """
    Determine the dew point temperature for air/water vapor.

    Parameters:
        p (float): Absolute pressure of the mixture [psia].
        T (float): Dry bulb temperature [F].
        r (float): Relative humidity [%].

    Returns:
        float: Dew point temperature [F].
    """
    # Initialize coefficients
    C = [0, 100.45, 33.193, 2.319, 0.17074, 1.2063]

    # Determine the partial pressure of water vapor in the air, [psia]
    pw = (p*W)/(0.621945 + W)
    alpha = math.log(pw)

    # Initial dew point temperature estimate
    Tdp_test = 90.12 + 26.142 * alpha + 0.8972 * alpha**2

    if Tdp_test < 32:
        T_dp =  Tdp_test
        return T_dp
    elif 32 <= Tdp_test <= 200:
        T_dp =  (C[1] + C[2] * alpha + C[3] * alpha**2 +
                C[4] * alpha**3 + C[5] * pw**0.1984)
        return T_dp
    else:
        raise ValueError("Dew point temperature out of valid range")